In [1]:
#%pwd
import os
from collections.abc import Iterable
from typing import Any, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm
from dotenv import load_dotenv
from IPython.display import HTML, display

In [2]:
import getpass
import os
from constant import api_key, watsonx_project_id

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": api_key
}

In [3]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings
#from langchain.llms import HuggingFaceHub
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

#print([model.name for model in ModelTypes])

In [4]:
model_id = ModelTypes.GRANITE_13B_CHAT_V2

In [5]:
model_id 

<ModelTypes.GRANITE_13B_CHAT_V2: 'ibm/granite-13b-chat-v2'>

In [6]:
model_id = ModelTypes.MT0_XXL
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}

from ibm_watson_machine_learning.foundation_models import Model

starcoder = Model(
    model_id=model_id, 
    params=parameters, 
    credentials=credentials,
    project_id=watsonx_project_id)
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

llm = WatsonxLLM(model=starcoder)

In [7]:
print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT']


In [8]:
print(dir(llm))

['Config', 'InputType', 'OutputType', '__abstractmethods__', '__annotations__', '__call__', '__class__', '__class_getitem__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__orig_bases__', '__parameters__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__ror__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '__weakref__', '_abatc

In [9]:
prompt = [
    """
    You are an expert in converting English questions to SQL query!
    The SQL database has the name STUDENT and has the following columns - NAME, CLASS, SECTION.
    For example:
    Example 1 - How many entries of records are present?, the SQL command will be: SELECT COUNT(*) FROM STUDENT;
    Example 2 - Tell me all the students studying in Data Science class?, the SQL command will be: SELECT * FROM STUDENT where CLASS='Data Science';
    Ensure the SQL code does not have ``` in the beginning or end and no SQL word in the output.
    """
]

In [10]:
prompt[0]

"\n    You are an expert in converting English questions to SQL query!\n    The SQL database has the name STUDENT and has the following columns - NAME, CLASS, SECTION.\n    For example:\n    Example 1 - How many entries of records are present?, the SQL command will be: SELECT COUNT(*) FROM STUDENT;\n    Example 2 - Tell me all the students studying in Data Science class?, the SQL command will be: SELECT * FROM STUDENT where CLASS='Data Science';\n    Ensure the SQL code does not have ``` in the beginning or end and no SQL word in the output.\n    "

In [11]:
question="provide query for average student marks"

In [12]:
response = llm.generate([prompt[0], question])

In [13]:
response

LLMResult(generations=[[Generation(text="SELECT * FROM STUDENT; SELECT * FROM STUDENT where NAME='NAME'; SELECT * FROM STUDENT where SECTION='SECTION'; SELECT * FROM STUDENT where NAME=NAME; SELECT * FROM STUDENT where SECTION=SECTION; SELECT * FROM STUDENT where NAME=NAME; SELECT * FROM STUDENT where SECTION=SECTION; SELECT * FROM")], [Generation(text='average_student_marks = np.mean(student_marks)')]], llm_output=None, run=[RunInfo(run_id=UUID('8195f7bb-0f7c-4527-9abf-de411dde5dc6')), RunInfo(run_id=UUID('08daa195-8c8d-4e1f-a97c-6e83cd19edd9'))])

In [14]:
response.generations[0][0].text

"SELECT * FROM STUDENT; SELECT * FROM STUDENT where NAME='NAME'; SELECT * FROM STUDENT where SECTION='SECTION'; SELECT * FROM STUDENT where NAME=NAME; SELECT * FROM STUDENT where SECTION=SECTION; SELECT * FROM STUDENT where NAME=NAME; SELECT * FROM STUDENT where SECTION=SECTION; SELECT * FROM"

In [15]:
# model works- granite_instruct_v2, flan_t5_xxl,flan_ul2,flan_t5_xl,mt0_xxl